## PROJECT SETUP & DATA OVERVIEW
 **0.1 Introduction & Business Context**

 ......
 .....

**0.2 Data Loading & Initial Exploration**

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import os

import statsmodels.formula.api as smf

## from
from matplotlib.colors import LinearSegmentedColormap
from statsmodels.tsa.seasonal import STL

import textwrap

#from sklearn.cluster import KMeans
#from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_columns', None)

In [ ]:
plt.style.use('default')

In [ ]:
df = pd.read_csv("../data/clean/aggregate_df.csv")

df_customers = pd.read_csv("../data/clean/customers.csv")
df_orders = pd.read_csv("../data/clean/orders.csv")
df_items = pd.read_csv("../data/clean/order_items.csv")
df_products = pd.read_csv("../data/clean/products.csv")

In [ ]:
print("=== MAIN DF ===")
df.info()
print("\n=== CUSTOMERS ===")
df_customers.info()
print("\n=== ORDERS ===")
df_orders.info()
print("\n=== ORDER ITEMS ===")
df_items.info()
print("\n=== PRODUCTS ===")
df_products.info()

In [ ]:
display(
    "=== CUSTOMERS ===",
    df_customers.describe().round(2).T,
    df_customers.shape,


    "=== ORDERS ===",
    df_orders.describe().round(2).T,
    df_orders.shape,

    "=== ORDER ITEMS ===",
    df_items.describe().round(2).T,
    df_items.shape,


    "=== PRODUCTS ===",
    df_products.describe().round(2).T,
    df_products.shape
    )



In [ ]:
### ---  Column structuring --- 

id_cols = ['Customers.id', 'Orders.id', 'Orders.customer_id', 'Orders.order_number', 'Orders.reorder_id',
           'Orders.coupon_id', 'Orders.reorder_id',
          'Order_Items.id', 'Order_Items.parent', 'Order_Items.product_id', 
          'Products.id',  'Products.import_id', 'Products.family_id'
        ]		


text_cols = ['Customers.fname', 'Customers.lname', 'Customers.company', 
            'Orders.company', 'Orders.discount_name', 'Orders.shipping_method', 
            'Order_Items.product_name', 
            'Products.shopping_color'
            ]

integer_cols = ['Customers.mailing', 'Order_Items.qty']


category_cols = ['Orders.shipping_carrier', 'Orders.shipping_method', 'Orders.discount_name',
                'Orders.payment_method', 'Orders.payment_status', 'Orders.status', 
                'Products.status', 'Products.vendor', 'Products.shopping_brand', 'Products.taxable'
                ]

date_cols = ['Orders.placed_date', 'Orders.payment_date', 'Orders.updated_date',
             'Orders.shipped_date', 'Customers.create_date', 'Customers.last_modified'
             ]

In [ ]:
dfs = [df, df_orders, df_customers, df_items, df_products]

for data in dfs:
    for col in id_cols:
        if col in data.columns:
            data[col] = data[col].astype("object")

    for col in category_cols:
        if col in data.columns:
            data[col] = data[col].astype("category")

    for col in integer_cols:
        if col in data.columns:
            data[col] = data[col].astype("int64")

    for col in date_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')



### **PART 1: CUSTOMER ANALYTICS**

**1.1 Customer Segmentation**

In [ ]:
metrics = {
    "Total Customers": df_customers['Customers.id'].nunique(),
    "Business Customers": (df_customers['Customers.company'] != "Individual").sum(),
    "Business Companies": df_customers['Customers.company'].nunique() - 1,
    "Individual Customers": (df_customers['Customers.company'] == "Individual").sum()
}

df_metrics = pd.DataFrame(list(metrics.items()), columns=["Category", "Count"])

# Desired order
order = [
    "Total Customers",
    "Individual Customers",
    "Business Customers",
    "Business Companies"
]

plt.figure(figsize=(8, 5))
sns.barplot(
    data=df_metrics,
    x="Category",
    y="Count",
    width=0.5,
    hue="Category",
    palette="Blues_d",
    order=order
)

# Add labels
for index, row in df_metrics.set_index("Category").loc[order].iterrows():
    plt.text(order.index(index), row["Count"] + 50, str(row["Count"]),
             ha='center', fontsize=11)

plt.ylim(0, 3400)
plt.title("Customer Overview")
plt.ylabel("Count")
plt.xlabel("")
plt.xticks(rotation=30, ha="right")
plt.tight_layout()
plt.show()

In [ ]:
df['is_individual'] = df['Customers.company'] == "Individual"
df['is_business'] = df['Customers.company'] != "Individual"

orders_comp_unique = (
    df
    .drop_duplicates(subset="Orders.id")
    .loc[:, ["Orders.id", "Orders.total", "is_individual", "is_business"]]
)

individual_revenue = orders_comp_unique.loc[orders_comp_unique['is_individual'], 'Orders.total'].sum()
business_revenue   = orders_comp_unique.loc[orders_comp_unique['is_business'], 'Orders.total'].sum()

print(f"Individual revenue: {individual_revenue:,.2f} USD")
print(f"Business revenue: {business_revenue:,.2f} USD")

In [ ]:
total_revenue = df_orders['Orders.total'].sum()

print("Individual share:", round((individual_revenue / total_revenue * 100),2), "%")
print("Business share:", round((business_revenue / total_revenue * 100),2), "%")

sizes = [individual_revenue, business_revenue]
labels = ["Individual", "Business"]

plt.figure(figsize=(5, 5))
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90,
        colors=["#66c2a5", "#3288bd"], wedgeprops={'width':0.4})

plt.title("B2B vs B2C distribution ")
plt.show()

In [ ]:
# ==================CUSTOMER DATA========================
df_customers['customer_type'] = np.where(df_customers['Customers.company'] == "Individual","Individual","Business")

customer_counts = df_customers['customer_type'].value_counts().sort_values()
customer_counts = customer_counts.reset_index()
customer_counts.columns = ['customer_type', 'customer_count']

# =================== ORDER DATA =======================

# Join customer type (temporary, for analysis only)
orders_with_type = df_orders.merge(df_customers[['Customers.id', 'is_business']],
    left_on='Orders.customer_id',
    right_on='Customers.id',
    how='left'
)
orders_with_type['customer_type'] = orders_with_type['is_business'].map(arg={False: 'Individual', True: 'Business'})

# ---- Mean order value ----
mean_values = (orders_with_type.groupby('customer_type')['Orders.total'].mean().reset_index())

# ---- Order count ----
order_counts = (orders_with_type.groupby('customer_type')['Orders.id'].count().reset_index(name='order_count'))

# =================== Total spent =======================

total_spent = (orders_with_type.groupby('customer_type')['Orders.total'].sum().reset_index(name='total_spent'))



# =================== PLOT =======================
fig, axes = plt.subplots(2, 2, figsize=(10, 7))

# === (1) CUSTOMER COUNT ===
sns.barplot(
    data=customer_counts,
    x='customer_type', y='customer_count',
    hue='customer_type', palette={'Individual': '#66c2a5', 'Business': '#3288bd'},
    legend=False,  width=0.4,
    ax=axes[0, 0]
)
axes[0, 0].set_title("Anzahl der Kunden")
axes[0, 0].set_xlabel("")
axes[0, 0].set_ylabel("Kundenanzahl")

# === (2) ORDER COUNT ===
sns.barplot(
    data=order_counts,
    x='customer_type', y='order_count',
    hue='customer_type', palette={'Individual': '#66c2a5', 'Business': '#3288bd'},
    legend=False, width=0.4,
    ax=axes[0, 1]
)
axes[0, 1].set_title("Anzahl der Bestellungen")
axes[0, 1].set_xlabel("")
axes[0, 1].set_ylabel("Bestellungen")

# === (3) MEAN ORDER VALUE ===
sns.barplot(
    data=mean_values,
    x='customer_type', y='Orders.total',
    hue='customer_type', palette={'Individual': '#66c2a5', 'Business': '#3288bd'},
    legend=False, width=0.4,
    ax=axes[1, 0]
)
axes[1, 0].set_title("Durchschnittlicher Bestellwert")
axes[1, 0].set_xlabel("Kundentyp")
axes[1, 0].set_ylabel("Bestellwert")

# === (4) TOTAL SPENT ===
sns.barplot(
    data=total_spent,
    x='customer_type', y='total_spent',
    hue='customer_type', palette={'Individual': '#66c2a5', 'Business': '#3288bd'},
    legend=False, width=0.4,
    ax=axes[1, 1]
)
axes[1, 1].set_title("Summe der Ausgaben (Revenue)")
axes[1, 1].set_xlabel("Kundentyp")
axes[1, 1].set_ylabel("Gesamtausgaben")

plt.tight_layout()
plt.show()

**1.2 RFM Analysis**

In [ ]:
df['Orders.placed_date'] = pd.to_datetime(df['Orders.placed_date'], errors='coerce')#прибрати!!!!
# ----------------------------
# 1. Reference date
# ----------------------------
reference_date = df['Orders.placed_date'].max()

# ----------------------------
# 2. Підготовка окремої таблиці замовлень без дублювання
# ----------------------------
orders_unique = df.drop_duplicates(subset='Orders.id').copy()

# Для коректності — гарантуємо правильні типи
orders_unique['Orders.total'] = pd.to_numeric(orders_unique['Orders.total'], errors='coerce')
orders_unique['Orders.placed_date'] = pd.to_datetime(orders_unique['Orders.placed_date'], errors='coerce')

# ----------------------------
# 3. Обчислення RFM
# ----------------------------

# Recency — коли клієнт востаннє зробив замовлення
recency = (reference_date - orders_unique.groupby('Customers.id')['Orders.placed_date'].max()).dt.days
recency = recency.rename('Recency_days')

# Frequency — кількість унікальних замовлень
frequency = orders_unique.groupby('Customers.id')['Orders.id'].nunique()
frequency = frequency.rename('Frequency_orders')

# Monetary — сума унікальних замовлень
monetary = orders_unique.groupby('Customers.id')['Orders.total'].sum()
monetary = monetary.rename('Monetary_total')

# ----------------------------
# 4. Об'єднання в RFM таблицю
# ----------------------------
rfm = pd.concat([recency, frequency, monetary], axis=1).reset_index()

print(rfm.head())
print(rfm.shape)

In [ ]:
# Recency bins - qcut OK
rfm['Recency_bin'] = pd.qcut(
    rfm['Recency_days'], 
    q=4,
    labels=['Low', 'Mid-Low', 'Mid-High', 'High'],
    duplicates='drop'
)

# Frequency bins – manual to avoid duplicates
rfm['Frequency_bin'] = pd.cut(
    rfm['Frequency_orders'],
    bins=[0, 1, 3, 6, 100],
    labels=['Low', 'Mid-Low', 'Mid-High', 'High']
)

# Pivot for heatmap
pivot = rfm.pivot_table(
    values='Monetary_total',
    index='Recency_bin',
    columns='Frequency_bin',
    aggfunc='mean'
)

plt.figure(figsize=(8, 6))
sns.heatmap(pivot, annot=True, fmt=".0f", cmap='YlGnBu')
plt.title("RFM Heatmap: Average Monetary Value")
plt.xlabel("Frequency Segment")
plt.ylabel("Recency Segment")
plt.show()

**1.3 Customer Lifetime Value**

In [ ]:
#Historical CLV
rfm['CLV'] = rfm['Monetary_total']

#CLV (High / Medium / Low) 0.33 та 0.66
rfm['CLV_segment'] = pd.qcut(rfm['CLV'], q=3, labels=['Low', 'Medium', 'High'])

In [ ]:
#at risk
# "high recency" = no bay long time -> risk 
# top 20% Recency

threshold = rfm['Recency_days'].quantile(0.80)
rfm['At_risk'] = (rfm['Recency_days'] > threshold).astype(int)


rfm['High_value_at_risk'] = ((rfm['CLV_segment'] == 'High') & 
                             (rfm['At_risk'] == 1)).astype(int)

In [ ]:
rfm_sorted = rfm.sort_values('CLV', ascending=False)
top20 = rfm_sorted.head(int(0.2 * len(rfm_sorted)))
top20_revenue_share = top20['CLV'].sum() / rfm['CLV'].sum()
print(f"Top 20% customers generate {top20_revenue_share:.2%} of revenue")

In [ ]:
lost_clv = rfm[rfm['High_value_at_risk'] == 1]['CLV'].sum()
print(f"At Risk segment has ${lost_clv:,.0f} CLV at stake")

In [ ]:
# Уникаємо попереджень
#sns.set(style="whitegrid")

# Кольори для CLV сегментів
palette = {
    'Low': '#a6cee3',
    'Medium': '#1f78b4',
    'High': '#b2df8a'
}

# --------------------------------------------
# 1️⃣ Recency vs Frequency
# --------------------------------------------
plt.figure(figsize=(8,6))
sns.scatterplot(
    data=rfm,
    x='Frequency_orders',
    y='Recency_days',
    hue='CLV_segment',
    palette=palette,
    alpha=0.7,
    s=60
)
plt.title("Recency vs Frequency (RFM Scatter)")
plt.xlabel("Frequency (Number of Orders)")
plt.ylabel("Recency (Days Since Last Order)")
plt.legend(title="CLV Segment")
plt.show()

# --------------------------------------------
# 2️⃣ Frequency vs Monetary
# --------------------------------------------
plt.figure(figsize=(8,6))
sns.scatterplot(
    data=rfm,
    x='Frequency_orders',
    y='Monetary_total',
    hue='CLV_segment',
    palette=palette,
    alpha=0.7,
    s=60
)
plt.title("Frequency vs Monetary")
plt.xlabel("Frequency (Number of Orders)")
plt.ylabel("Monetary (Total Spend, USD)")
plt.legend(title="CLV Segment")
plt.show()

# --------------------------------------------
# 3️⃣ Recency vs Monetary (highlight high-value)
# --------------------------------------------
plt.figure(figsize=(8,6))
sns.scatterplot(
    data=rfm,
    x='Recency_days',
    y='Monetary_total',
    hue='CLV_segment',
    palette=palette,
    alpha=0.7,
    size='Monetary_total',
    sizes=(20, 200),
    legend="brief"
)
plt.title("Recency vs Monetary (Bubble Plot)")
plt.xlabel("Recency (Days Since Last Order)")
plt.ylabel("Monetary (Total Spend, USD)")
plt.legend(title="CLV Segment")
plt.show()

**1.4 Cohort Retention Analysis**

### **PART 2: ORDER & REVENUE ANALYTICS**

**2.1 Order Distribution**

In [ ]:
plt.figure(figsize=(12, 10))

# =================== 1. Histogram: all Orders.total 
plt.subplot(3, 1, 1)
sns.histplot(df_orders['Orders.total'], bins=40)
plt.title('Histogram: Orders.total (All)')

# =================== 2. Boxplot: all Orders.total 
plt.subplot(3, 1, 2)
sns.boxplot(x=df_orders['Orders.total'])
plt.title('Boxplot: Orders.total (All)')

# =================== 3. Histogram: Orders.total <= 400 
plt.subplot(3, 1, 3)
sns.histplot(df_orders[df_orders['Orders.total'] <= 400]['Orders.total'], bins=100)
plt.title('Histogram: Orders.total (≤ 400)')

plt.tight_layout()
plt.show()

In [ ]:
### --- outliers investigation --- ORDERs!!!!!
#Filtered High-Value Orders ( >1500€ )

#df_orders[df_orders['Orders.total'] > 1500].sort_values(by='Orders.total', ascending=False)

print("----------------------------------------------------------------------\n")
print("                Hochwertige Bestellungen (über 1500 $)       \n")

df_orders[df_orders['Orders.total'] > 1500] \
    .sort_values(by='Orders.total', ascending=False) \
    [['Orders.id', 'Orders.customer_id', 'Orders.total', 'Orders.payment_amount' ]]


**2.2 Order Status Analysis**

In [ ]:
print("-------------------------------------------------------\n")
print("               Bestellstatus nach Zahlungsstatus  \n")


ct_status = pd.crosstab(df_orders["Orders.status"], df_orders["Orders.payment_status"])
ct_status

In [ ]:
df_orders["has_reorder"] = df_orders["Orders.reorder_id"].notna().astype(int)
df_orders["has_tax"] = (df_orders["Orders.tax"] > 0).astype(int)
df_orders["has_shipping"] = (df_orders["Orders.shipping"] > 0).astype(int)
df_orders["has_coupon"] = df_orders["Orders.coupon_id"].notna().astype(int)

# discount_name є завжди, але треба перевірити, чи там "No Discount", "" або NaN
df_orders["has_discount"] = df_orders["Orders.discount_name"].ne("No Discount").astype(int)

In [ ]:
print("----------------------------------------------------------------------------\n")
print("                      Orders.status  nach  Feature-Indikatoren  \n")

ct_features = df_orders.groupby("Orders.status", observed=True)[[
    "has_reorder",
    "has_tax",
    "has_shipping",
    "has_coupon",
    "has_discount"
]].sum()

ct_features

In [ ]:
# # ---------- 1. Бінарні колонки ----------
# df_orders["has_reorder"] = df_orders["Orders.reorder_id"].notna().astype(int)
# df_orders["has_tax"] = (df_orders["Orders.tax"] > 0).astype(int)
# df_orders["has_coupon"] = df_orders["Orders.coupon_id"].notna().astype(int)
# df_orders["has_discount"] = df_orders["Orders.discount_name"].ne("No Discount").astype(int)

# # ---------- 2. Крос-таблиця ----------
# ct_features = df_orders.groupby("Orders.status")[[
#     "has_reorder",
#     "has_tax",
#     "has_shipping",
#     "has_coupon",
#     "has_discount"
# ]].sum()


# ---------- 3. Heatmap ----------
from matplotlib.colors import LinearSegmentedColormap

colors = [
    "#fdf4f4",  # 0%
    "#D34646",  # 10%
    "#970000",  # 25%
    "#523B3B",  
    "#523B3B",  # 50%
    "#523B3B", 
    "#523B3B",  # 75%
    "#523B3B",   
    "#523B3B",  # 100%         
]

custom_cmap = LinearSegmentedColormap.from_list("custom_heatmap", colors)

plt.figure(figsize=(8, 5))
sns.heatmap(
    ct_features,
    annot=True,
    fmt="d",
    cmap=custom_cmap,
    linewidths=0.5,
    linecolor="white"
)

plt.title("Feature-Heatmap nach Bestellstatus")
plt.ylabel("Orders.status")
plt.tight_layout()
plt.show()

**2.3 Temporal Trends**

In [ ]:
df_orders['Orders.placed_date'] = pd.to_datetime(df_orders['Orders.placed_date'])

daily_sales = df_orders.groupby('Orders.placed_date')['Orders.total'].sum().sort_index()
weekly_sales = df_orders.resample('W', on='Orders.placed_date')['Orders.total'].sum()
monthly_sales = df_orders.resample('M', on='Orders.placed_date')['Orders.total'].sum()

In [ ]:
plt.figure(figsize=(12,4))
daily_sales.plot(label='Daily Sales', alpha=0.5)
daily_sales.rolling(7).mean().plot(label='7-day MA', linewidth=2)
daily_sales.rolling(30).mean().plot(label='30-day MA', linewidth=2)
plt.legend()
plt.title("Revenue Trend with Moving Averages")
plt.grid(alpha=0.3)
plt.show()

Moving averages демонструють чіткий тренд зростання продажів протягом 2014–2016 років.
Щоденні продажі характеризуються високою волатильністю через оптові замовлення, але 7-денне і 30-денне середнє показують стабільне збільшення середнього доходу.

In [ ]:
df_orders['Orders.placed_date'] = pd.to_datetime(df_orders['Orders.placed_date'])

daily = df_orders.groupby(df_orders['Orders.placed_date'].dt.date)['Orders.total'].sum()
daily.index = pd.to_datetime(daily.index)

In [ ]:
#from statsmodels.tsa.seasonal import STL

stl = STL(daily, period=7)
res = stl.fit()
res.plot()
plt.show()

Підсумок у 4 реченнях (в проект.md)

Дані показують чіткий зростаючий тренд продажів протягом 2014–2016 років.
Присутня стабільна тижнева сезонність, яка посилюється з часом.
Велика кількість аномальних днів пов’язана з великими B2B-замовленнями.
Ряд добре піддається трендовому аналізу, але важко моделюється ARIMA через сильні аномалії.

In [ ]:
# --- Monthly Revenue from df_orders ---

df_orders['Orders.placed_date'] = pd.to_datetime(df_orders['Orders.placed_date'])

# Create year-month column
df_orders['year_month'] = df_orders['Orders.placed_date'].dt.to_period('M').astype(str)

# Aggregate revenue
monthly_revenue = (
    df_orders.groupby('year_month')['Orders.total']
    .sum()
    .reset_index()
)

# Plot
plt.figure(figsize=(14, 5))
sns.barplot(
    data=monthly_revenue,
    x='year_month',
    y='Orders.total',
    color='#4C72B0'
)

plt.xticks(rotation=90)
plt.title("Monthly Revenue (Total Sales per Month)")
plt.xlabel("Year-Month")
plt.ylabel("Revenue (USD)")
plt.grid(axis='y', alpha=0.3)

plt.show()


### **PART 3: PAYMENT & SHIPPING OPERATIONS**

**3.1 Payment Analysis**

In [ ]:
print("--------------------------------------------------------\n")
print("              Zahlungsart nach Zahlungsstatus  \n")
ct_pay_method = pd.crosstab(df_orders["Orders.payment_method"], df_orders["Orders.payment_status"])
ct_pay_method

In [ ]:
# --- Єдиний порядок payment_method для графіків 3 і 4 ---
method_order = sorted(df_orders['Orders.payment_method'].dropna().unique())


fig, axes = plt.subplots(2, 2, figsize=(18, 12))

# === 1. Payment Status Distribution ===
sns.countplot(
    data=df_orders,
    x='Orders.payment_status',
    order=df_orders['Orders.payment_status'].value_counts().index,
    ax=axes[0, 0],
    color="#1A5782"
 
)
axes[0, 0].set_title('Payment Status Distribution', fontsize=14)
axes[0, 0].set_xlabel('Payment Status')
axes[0, 0].set_ylabel('Count')
axes[0, 0].tick_params(axis='x', rotation=0)


# === 2. Heatmap (Payment Status by Payment Method) ===
ct = pd.crosstab(
    df_orders['Orders.payment_method'],
    df_orders['Orders.payment_status'],
    normalize='index'
) * 100

sns.heatmap(
    ct,
    annot=True,
    fmt=".1f",
    cmap='PuBu',
    linewidths=.5,
    ax=axes[0, 1]
)

axes[0, 1].set_title('Payment Status by Payment Method (%)', fontsize=14)
axes[0, 1].set_xlabel('Payment Status')
axes[0, 1].set_ylabel(' ')


# === 3. Payment Method Distribution (HORIZONTAL, sorted!) ===
sns.countplot(
    data=df_orders,
    y='Orders.payment_method',     # ← категорії тепер по осі Y
    order=method_order,
    ax=axes[1, 0],
    palette="viridis",
)
axes[1, 0].set_title('Payment Method Distribution', fontsize=14)
axes[1, 0].set_xlabel('Count')
axes[1, 0].set_ylabel('Payment Method')
axes[1, 0].tick_params(axis='y', rotation=0)


# === 4. Average Order Total by Payment Method (HORIZONTAL) ===
sns.barplot(
    data=df_orders,
    y='Orders.payment_method',     # ← теж по Y
    x='Orders.total',
    estimator='mean',
    order=method_order,
    ax=axes[1, 1],
    palette="viridis",
    alpha=0.9
)
axes[1, 1].set_title('Average Order Total by Payment Method', fontsize=14)
axes[1, 1].set_xlabel('Average Order Total (€)')
axes[1, 1].set_ylabel('Payment Method')
axes[1, 1].tick_params(axis='y', rotation=0)

plt.tight_layout()
plt.show()


confidence intervals - довірчі інтервали які seaborn додає за замовчуванням до barplot.
Вони показують:
Наскільки надійна оцінка середнього значення
Наявність варіації / розкиду даних в групі
Якщо лінія дуже довга - дані сильно варіюються (великий розкид значень)
Якщо коротка - середня оцінка стабільна.

authorize.net  - payment gateway in USA ()

**3.2 Shipping & Logistics**

In [ ]:
df_clean = df_orders.dropna(subset=[
    "Orders.shipping_carrier",
    "Orders.shipping_method"
])

df_clean["shipping_paid"] = (df_clean["Orders.shipping"] > 0).astype(int)

ct_carrier = pd.crosstab(df_clean["Orders.shipping_carrier"],
                          df_clean["shipping_paid"]
)
ct_carrier

In [ ]:
ct_carrier = pd.crosstab(
    df_clean["Orders.shipping_carrier"],
    df_clean["shipping_paid"]
)

main_methods = ["fedex", "manual", "ups"]
ct_main = ct_carrier.loc[main_methods]

free = ct_main[0].values
paid = ct_main[1].values
methods = ct_main.index.tolist()

x = np.arange(len(methods))
width = 0.3

plt.figure(figsize=(7, 5))

plt.bar(x - width/2, free, width, label="Free", color="#E97474")
plt.bar(x + width/2, paid, width, label="Paid", color="#523B3B")

plt.xticks(x, methods)
plt.ylabel("Number of Orders")
plt.xlabel("Shipping Carrier")
plt.title("Shipping Carrier: Free vs Paid Delivery")
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
ct_method = pd.crosstab(
    df_clean["shipping_paid"],
    df_clean["Orders.shipping_method"]
)
ct_method.T

In [ ]:
plt.figure(figsize=(8, 5))

sns.scatterplot(
    data=df_orders,
    x="Orders.total",
    y="Orders.shipping",
   # color="#208989",
    s=80,        # більші точки
    alpha=0.5    # більше прозорості
)

plt.xlabel("Order Total (€)")
plt.ylabel("Shipping Cost (€)")
plt.title("Verhältnis zwischen Bestellwert und Versandkosten")
plt.grid(True, alpha=0.3)
# Achsenbereich auf der Y-Achse so begrenzen, dass nur die kostenpflichtigen Lieferungen angezeigt werden (>0)
#plt.ylim(1,35)
plt.show()

### **PART 4: PRODUCT & PROFITABILITY ANALYTICS**

**4.1 Product Distribution**

In [ ]:
colors = sns.color_palette("PuBu", n_colors=6)[1:]

fig, axes = plt.subplots(1, 2, figsize=(10, 4))

# -------------------------------
# 1) Кількість товарів у замовленні (1–5)
# -------------------------------
small_qty = df[df['Order_Items.qty'] <= 5]

small_qty['Order_Items.qty'] \
    .value_counts() \
    .sort_index() \
    .plot(kind='bar', color=colors, ax=axes[0])

axes[0].set_title('Number of Items in an Order (1 - 5)')
axes[0].set_xlabel('Quantity')
axes[0].set_ylabel('Count')
axes[0].tick_params(axis='x', rotation=0)

# -------------------------------
# 2) Частота повторного замовлення
# -------------------------------
(df_items[df_items['Order_Items.reorder_frequency'].notna()]
    ['Order_Items.reorder_frequency']
    .value_counts()
    .sort_index()
    .plot(kind='bar', color=colors, ax=axes[1]))

axes[1].set_title('Reorder Frequency Distribution')
axes[1].set_xlabel('Reorder Frequency')
axes[1].set_ylabel('Count')
axes[1].tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.show()


**4.2 Pricing & Margin Analysis**

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(10, 7), sharex=False)

price_color = "#4472C4"   # blue
cost_color  = "#FFD966"   # yellow


# ===== TOP subplot: ≤ 200$ =====
sns.histplot(
    df_products[(df_products["Products.price"].notna()) &
                (df_products["Products.price"] <= 200)],
    x="Products.price",
    bins=50,
    color=price_color,
    alpha=0.45,
    edgecolor="white",
    linewidth=0.6,
    ax=axes[0]
)

sns.histplot(
    df_products[(df_products["Products.cost"].notna()) &
                (df_products["Products.cost"] <= 200)],
    x="Products.cost",
    bins=50,
    color=cost_color,
    alpha=0.45,
    edgecolor="white",
    linewidth=0.6,
    ax=axes[0]
)

axes[0].set_title("Price vs Cost (≤ 200$)", fontsize=15, pad=12)
axes[0].set_xlabel("Value ($)", fontsize=12)
axes[0].set_ylabel("Count", fontsize=12)
axes[0].grid(axis='y', alpha=0.25)
axes[0].legend(["Price ≤ 200$", "Cost ≤ 200$"])


# ===== BOTTOM subplot: > 200$ =====
sns.histplot(
    df_products[(df_products["Products.price"].notna()) &
                (df_products["Products.price"] > 200)],
    x="Products.price",
    bins=50,
    color=price_color,
    alpha=0.45,
    edgecolor="white",
    linewidth=0.6,
    ax=axes[1]
)

sns.histplot(
    df_products[(df_products["Products.cost"].notna()) &
                (df_products["Products.cost"] > 200)],
    x="Products.cost",
    bins=50,
    color=cost_color,
    alpha=0.45,
    edgecolor="white",
    linewidth=0.6,
    ax=axes[1]
)

axes[1].set_title("Price vs Cost (> 200$)", fontsize=15, pad=12)
axes[1].set_xlabel("Value ($)", fontsize=12)
axes[1].set_ylabel("Count", fontsize=12)
axes[1].grid(axis='y', alpha=0.25)
axes[1].legend(["Price > 200$", "Cost > 200$"])


plt.tight_layout()
plt.show()


In [ ]:
df_products["margin"] = df_products["Products.price"] - df_products["Products.cost"]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
ax1, ax2 = axes


# === 1: Scatter: Margin vs Cost ===
sns.scatterplot(
    data=df_products,
    x="Products.cost",
    y="margin",
    alpha=0.5,
   # color="#4472C4",
    ax=ax1
)
ax1.axhline(0, color="red", linestyle="--")
ax1.set_title("Product Margin vs Cost", fontsize=14)
ax1.set_xlabel("Cost ($)")
ax1.set_ylabel("Margin ($)")
ax1.grid(alpha=0.3)


# === 2: Line plot: Sorted Margin Curve ===
#diff_sorted = df_products["margin"].dropna().sort_values().reset_index(drop=True)

sns.boxplot(
    x=df_products["margin"],
    ax=ax2
)

ax2.set_title("Product Margin Distribution", fontsize=14)
ax2.set_xlabel("Margin ($)")


plt.tight_layout()
plt.show()

**4.3 Top Performers**

In [ ]:
top_margin = (
    df_products[["Products.name", "Products.price", "Products.cost", "margin", "margin_perc"]]
    .sort_values(by="margin", ascending=False)
    .head(10)
)

top_margin.sort_values('margin')

In [ ]:
# 1 = реалізований, 0 = ні
realized_statuses = ['Completed','Shipped']
df['order_realized'] = (df['Orders.status'].isin(realized_statuses)).astype(int)

# item_margin
df['item_margin'] = (df['Order_Items.price'] - df['Order_Items.cost']) * df['Order_Items.qty']

# топ-10 продуктів за сумарним обсягом продажів ($)
top_products_realized = (
    (df[df['order_realized'] == 1])
    .groupby(['Products.id', 'Products.name'], as_index=False)
    .agg(
        total_sales=('Order_Items.price', 'sum'),
        quantity_sold=('Order_Items.qty', 'sum'),
        total_margin=('item_margin', 'sum')   # зароблена маржа
    )
    .sort_values('total_sales', ascending=False)
)

top_products_realized.head(10)

In [ ]:
top_margin_10 = top_margin.sort_values('margin')
top_marginsales_10 = top_products_realized.head(10).sort_values('total_margin')

#import textwrap
def wrap_labels(labels, width=28):
    return [textwrap.fill(label, width) for label in labels]


fig, axes = plt.subplots(1, 2, figsize=(18, 7))

# ---------- LEFT: Top Margin ----------
axes[0].barh(
    wrap_labels(top_margin_10['Products.name']),
    top_margin_10['margin'],
    color="#4C72B0"
)
axes[0].set_title('Top 10 Products by Absolute Margin', fontsize=13)
axes[0].set_xlabel('Margin ($)')
axes[0].set_ylabel('')
axes[0].grid(axis='x', alpha=0.3)

# ---------- RIGHT: Top Sales ----------
axes[1].barh(
    wrap_labels(top_marginsales_10['Products.name']),
    top_marginsales_10['total_margin'],
    color="#55A868"
)
axes[1].set_title('Top 10 Products by Cumulative Margin (Realized Orders)', fontsize=13)
axes[1].set_xlabel('Total Margin ($)')
axes[1].set_ylabel('')
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

**4.4 Vendor Analysis**

In [ ]:
cr_vendor_prod = pd.crosstab(df_products['Products.status'], 
                       df_products['Products.vendor'],
                       normalize='index') * 100
#cr_vendor_prod.round(1)


plt.figure(figsize=(8, 5))
sns.heatmap(
    cr_vendor_prod.T, 
    cmap="PuBu", 
    annot=True, 
    fmt=".1f"
)

plt.title("Products Status vs Vendor (%)", fontsize=14)
plt.xlabel("")
plt.ylabel("")
plt.tight_layout()
plt.show()

### **PART 5: PREDICTIVE ANALYTICS & MODELING**

**5.1 Margin Prediction Models**

In [ ]:
model_vend = smf.ols(
    formula = '''
        Q("margin") ~
        C(Q("Products.vendor"))
    ''',
    data=df_products
)

results_vend = model_vend.fit()
print(results_vend.summary())

In [ ]:
threshold = df_products['margin'].quantile(0.90)

df_products['high_margin'] = (df_products['margin'] >= threshold).astype(int)

print(df_products['high_margin'].value_counts())

# plt.figure(figsize=(4, 3))
# sns.countplot(data=df_products, x='high_margin')
# plt.title("Count of Products by Margin Segment")
# plt.xlabel("High Margin (1 = Top segment)")
# plt.show()

In [ ]:
logit_model = smf.logit(
    formula='''
        high_margin ~
        Q("Products.price") +
        C(Q("Products.vendor")) 

    ''',
    data=df_products
)

logit_results = logit_model.fit()
print(logit_results.summary())

In [ ]:
df_products['prob_high_margin'] = logit_results.predict(df_products)

plt.figure(figsize=(6, 4))

sns.scatterplot(
    data=df_products,
    x='margin',
    y='prob_high_margin',
    alpha=0.4
)

plt.axhline(0.5, linestyle='--')
plt.xlabel('Margin ($)')
plt.ylabel('Predicted Probability (High Absolute Margin)')
plt.title('Probability of High-Margin Products (Absolute $)')
plt.show()

**5.2 Reorder Prediction Models**

In [ ]:
#import statsmodels.formula.api as smf
# 

rfm['made_reorder'] = (rfm['Frequency_orders'] > 1).astype(int)

#додаємо бізнес ознаку

rfm = rfm.merge(
    df_customers[['Customers.id', 'is_business']],
    on='Customers.id',
    how='left'
) 

In [ ]:
model2 = smf.logit(
    formula='made_reorder ~ Recency_days + Monetary_total',
    data=rfm
)

results2 = model2.fit()
print(results2.summary())

In [ ]:
model3 = smf.logit(
    formula='made_reorder ~ Recency_days + Monetary_total + is_business',
    data=rfm
)
results3 = model3.fit()
print(results3.summary())

In [ ]:
mean_monetary = rfm['Monetary_total'].mean()
mean_business = rfm['is_business'].mean()

In [ ]:
X = pd.Series(range(rfm['Recency_days'].min(), rfm['Recency_days'].max() + 1))

X_df = pd.DataFrame({
    'Recency_days': X,
    'Monetary_total': mean_monetary,
    'is_business': mean_business
})

In [ ]:
p_y = results3.predict(X_df) 

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))
ax.plot(X, p_y, color='blue')

ax.set_title("Logistic regression curve: Recency effect")
ax.set_xlabel("Recency_days")
ax.set_ylabel("Probability of reorder")
ax.grid(alpha=0.3)

plt.show()

### **PART 6: BUSINESS INSIGHTS & RECOMMENDATIONS**

6.1 Key Findings Summary

6.2 Actionable Recommendations

6.3 Expected Business Impact

6.4 Next Steps

In [ ]:
###

In [ ]:
model_results = pd.DataFrame({
    'Model': [
        'OLS: Margin ~ Vendor',
        'Logistic: High-Margin ~ Vendor',
        'Logistic: Reorder ~ Recency',
        'Logistic: Reorder ~ Recency + Monetary',
        'Logistic: Reorder ~ Full Model'
    ],
    'R² / Pseudo R²': [
        0.058,
        0.75,
        0.007,
        0.154,
        0.21
    ],
    'Interpretation': [
        'Vendor explains little variation in margin',
        'Vendor strongly predicts high-margin products',
        'Recency alone has almost no predictive power',
        'Monetary value significantly improves prediction',
        'Business customers reorder more frequently'
    ]
})
model_results